This notebook demonstrates the result of the first round of data collection, for the Mountain View library <-> UC Berkeley round trip. The big flaw of this trip is that 

## Import all the dependencies

In [ ]:
# for reading and validating data
import emeval.input.spec_details as eisd
import emeval.input.phone_view as eipv
import emeval.input.eval_view as eiev

In [ ]:
# Visualization helpers
import emeval.viz.phone_view as ezpv
import emeval.viz.eval_view as ezev

In [ ]:
# For plots
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
# For maps
import branca.element as bre

## Load and validate data

The first issue to note is that we actually have two specs here. The first spec is the checked in `evaluation.spec.sample`, which defines calibration for both stationary and moving instances, and some evaluation trips. However, while starting with the calibration, we noticed some inconsistencies between the power curves. So in order to be more consistent, I defined a second, calibration-only spec `examples/calibration.only.json`, which essentially repeats the calibration experiments multiple times.

After that, I returned to the first set of experiments for the moving calibration and the evaluation.

In [ ]:
DATASTORE_URL = "http://cardshark.cs.berkeley.edu"
AUTHOR_EMAIL = "shankari@eecs.berkeley.edu"
sdunp = eisd.SpecDetails(DATASTORE_URL, AUTHOR_EMAIL, "train_bus_ebike_mtv_ucb")

In [ ]:
import importlib
importlib.reload(ezev)

In [ ]:
pvunp = eipv.PhoneView(sdunp)

In [ ]:
pvunp.validate()

In [ ]:
evunp = eiev.EvaluationView()
evunp.from_view_eval_trips(pvunp, "", "")

In [ ]:
[r["trip_id"] for r in pvunp.map()["ios"]["ucb-sdb-ios-2"]["evaluation_ranges"]]

In [ ]:
importlib.reload(ezev)

In [ ]:
map_list = ezev.get_map_list_eval_trips(evunp, "android", "", "HAHFDC|HAMFDC")
rows = ezpv.get_row_count(len(map_list), 2)
evaluation_maps = bre.Figure(ratio="{}%".format((rows/4) * 100))
for i, curr_map in enumerate(map_list):
    evaluation_maps.add_subplot(rows, 2, i+1).add_child(curr_map)

In [ ]:
evaluation_maps

In [ ]:
map_list = ezev.get_map_list_eval_trips(evunp, "ios", "", "HAHFDC|MAHFDC")
rows = ezpv.get_row_count(len(map_list), 2)
evaluation_maps = bre.Figure(ratio="{}%".format((rows/4) * 100))
for i, curr_map in enumerate(map_list):
    evaluation_maps.add_subplot(rows, 2, i+1).add_child(curr_map)

In [ ]:
evaluation_maps

In [ ]:
(ifig, [android_ax, ios_ax]) = plt.subplots(ncols=1, nrows=2, figsize=(10,10))

ezpv.plot_all_power_drain(ios_ax, pvunp.map()["ios"], "evaluation", "")
ios_ax.legend(loc="center left", bbox_to_anchor=(1, 0.5), ncol=1)
ezpv.plot_all_power_drain(android_ax, pvunp.map()["android"], "evaluation", "")
android_ax.legend(loc="center left", bbox_to_anchor=(1, 0.5), ncol=1)

In [ ]:
importlib.reload(ezpv)

In [ ]:
android_density_df = ezpv.get_location_density_df(pvunp.map()["android"], "evaluation")
nRows = ezpv.get_row_count(len(android_density_df), 2)
print(nRows)
android_ax = android_density_df.plot(kind='density', subplots=False, layout=(nRows, 2), figsize=(8,8), sharex=True, sharey=True)
android_ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))


In [ ]:
import numpy as np

In [ ]:
invalid_point_mask = np.full(len(android_density_df), False)
for col in android_density_df:
    before_points = android_density_df[col] < 0 # 6 minutes
    if np.count_nonzero(np.array(before_points)) > 0:
        print(col, android_density_df[col][before_points])
        invalid_point_mask = invalid_point_mask | before_points
        
print(np.nonzero(np.array(invalid_point_mask)))

In [ ]:
filtered_android_density_df = android_density_df[np.logical_not(invalid_point_mask)]
android_ax = filtered_android_density_df.plot(kind='density', subplots=False, layout=(nRows, 2), figsize=(8,8), sharex=True, sharey=True)
android_ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

In [ ]:
ios_density_df = ezpv.get_location_density_df(pvunp.map()["ios"], "evaluation")
nRows = ezpv.get_row_count(len(ios_density_df), 2)
print(nRows)
ios_ax = ios_density_df.plot(kind='density', subplots=False, layout=(nRows, 2), figsize=(10,10), sharex=True, sharey=True)
ios_ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))


In [ ]:
for col in ios_density_df:
    before_points = ios_density_df[col] < 0
    if len(before_points) > 0:
        print(col, ios_density_df[col][before_points])

In [ ]:
# Enable only if the previous step has any weird negative points, and modify the column
# col_with_weird_negative_points = "ucb-sdb-ios-2_HAHFDC v/s HAMFDC:HAHFDC_0"
# curr_col = ios_density_df[col_with_weird_negative_points]
# filtered_ios_density_df = ios_density_df[curr_col > 0].drop("ucb-sdb-ios-4_fixed:POWER_CONTROL_0", axis=1)
# ios_ax = filtered_ios_density_df.plot(kind='density', subplots=False, layout=(nRows, 2), figsize=(8,8), sharex=True, sharey=True)
# ios_ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

### Checking the motion activity

In addition to location data, we also read the motion_activity data from the closed source phone APIs. Let's quickly check how accurate the raw motion activity is.

The medium accuracy runs seem to be much more noisy wrt motion activity transitions. We should really not get a lot of transitions since we essentially took various trains for the entire route. The high accuracy sensing seems to be largely stable, except for one extraneous transition in the middle of the `ucb-sdb-ios-1` run.